<a href="https://colab.research.google.com/github/somilasthana/MachineLearningSkills/blob/master/Scratch_LogisticRegression_Using_Newton_Method.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%matplotlib inline

#matrix math
import numpy as np
#data manipulation
import pandas as pd
#matrix data structure
from patsy import dmatrices
#for error logging
import warnings

In [0]:
def sigmoid(x):
  return 1.0 / (1.0 + np.exp(-x))

In [0]:
np.random.seed(42)

#the minimum threshold for the difference between the predicted output and the actual output
#this tells our model when to stop learning, when our prediction capability is good enough
tol=1e-8 # convergence tolerance

lam = None


max_iter = 20 # maximum allowed iterations


r = 0.95 # covariance between x and z
n = 1000 # number of observations (size of dataset to generate) 
sigma = 1 # variance of noise - how spread out is the data?


In [0]:
## model settings
beta_x, beta_z, beta_v = -4, .9, 1 # true beta coefficients
var_x, var_z, var_v = 1, 1, 4 # variances of inputs

## the model specification you want to fit
formula = 'y ~ x + z + v + np.exp(x) + I(v**2 + z)'

In [0]:
#The multivariate normal, multinormal or Gaussian distribution is a generalization of the one-dimensional normal 

#lets keep x and z closely related (height and weight)
x, z = np.random.multivariate_normal([0,0], [[var_x,r],[r,var_z]], n).T


In [0]:
#blood presure
v = np.random.normal(0,var_v,n)**3

In [7]:
#create a pandas dataframe (easily parseable object for manipulation)
A = pd.DataFrame({'x' : x, 'z' : z, 'v' : v})
A.head()

,x,z,v
0,-0.468604,-0.512327,-19.698600
1,-0.880353,-0.398729,-0.193175
2,0.268228,0.194188,-31.845358
3,-1.680690,-1.438006,-1.869259
4,0.377783,0.549355,-434.565055


In [8]:
#compute the log odds for our 3 independent variables
#using the sigmoid function 
A['log_odds'] = sigmoid(A[['x','z','v']].dot([beta_x,beta_z,beta_v]) + sigma*np.random.normal(0,1,n))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: RuntimeWarning: overflow encountered in exp
  


In [9]:
A.head()

,x,z,v,log_odds
0,-0.468604,-0.512327,-19.698600,1.699241e-09
1,-0.880353,-0.398729,-0.193175,8.917746e-01
2,0.268228,0.194188,-31.845358,3.981542e-15
3,-1.680690,-1.438006,-1.869259,9.957092e-01
4,0.377783,0.549355,-434.565055,1.177472e-189


In [0]:
A['y'] = [np.random.binomial(1,p) for p in A.log_odds]

In [11]:
A.head()

,x,z,v,log_odds,y
0,-0.468604,-0.512327,-19.698600,1.699241e-09,0
1,-0.880353,-0.398729,-0.193175,8.917746e-01,1
2,0.268228,0.194188,-31.845358,3.981542e-15,0
3,-1.680690,-1.438006,-1.869259,9.957092e-01,1
4,0.377783,0.549355,-434.565055,1.177472e-189,0


In [0]:
y, X = dmatrices(formula, A, return_type='dataframe')

In [13]:
X.head()

,Intercept,x,z,v,np.exp(x),I(v ** 2 + z)
0,1.0,-0.468604,-0.512327,-19.698600,0.625875,387.522499
1,1.0,-0.880353,-0.398729,-0.193175,0.414636,-0.361412
2,1.0,0.268228,0.194188,-31.845358,1.307646,1014.320982
3,1.0,-1.680690,-1.438006,-1.869259,0.186245,2.056122
4,1.0,0.377783,0.549355,-434.565055,1.459046,188847.336466


In [0]:
def catch_singularity(f):
    '''Silences LinAlg Errors and throws a warning instead.'''
    
    def silencer(*args, **kwargs):
        try:
            return f(*args, **kwargs)
        except np.linalg.LinAlgError:
            warnings.warn('Algorithm terminated - singular Hessian!')
            return args[0]
    return silencer

In [0]:
@catch_singularity
def newton_step(curr, X, lam=None):
    '''One naive step of Newton's Method'''
    
    #how to compute inverse? http://www.mathwarehouse.com/algebra/matrix/images/square-matrix/inverse-matrix.gif
    
    ## compute necessary objects
    #create probability matrix, miniminum 2 dimensions, tranpose (flip it)
    p = np.array(sigmoid(X.dot(curr[:,0])), ndmin=2).T
    #create weight matrix from it
    W = np.diag((p*(1-p))[:,0])
    #derive the hessian 
    hessian = X.T.dot(W).dot(X)
    #derive the gradient
    grad = X.T.dot(y-p)
    
    ## regularization step (avoiding overfitting)
    if lam:
        # Return the least-squares solution to a linear matrix equation
        step, *_ = np.linalg.lstsq(hessian + lam*np.eye(curr.shape[0]), grad)
    else:
        step, *_ = np.linalg.lstsq(hessian, grad)
        
    ## update our 
    beta = curr + step
    
    return beta

In [0]:
def check_coefs_convergence(beta_old, beta_new, tol, iters):
    '''Checks whether the coefficients have converged in the l-infinity norm.
    Returns True if they have converged, False otherwise.'''
    #calculate the change in the coefficients
    coef_change = np.abs(beta_old - beta_new)
    
    #if change hasn't reached the threshold and we have more iterations to go, keep training
    return not (np.any(coef_change>tol) & (iters < max_iter))

In [17]:
beta_old, beta = np.ones((len(X.columns),1)), np.zeros((len(X.columns),1))

#num iterations we've done so far
iter_count = 0
#have we reached convergence?
coefs_converged = False

#if we haven't reached convergence... (training step)
while not coefs_converged:
    
    #set the old coefficients to our current
    beta_old = beta
    #perform a single step of newton's optimization on our data, set our updated beta values
    beta = newton_step(beta, X, lam=lam)
    #increment the number of iterations
    iter_count += 1
    
    #check for convergence between our old and new beta values
    coefs_converged = check_coefs_convergence(beta_old, beta, tol, iter_count)
    
print('Iterations : {}'.format(iter_count))
print('Beta : {}'.format(beta))

Iterations : 18
Beta : [[ 3.31473859e+20]
 [-8.62368597e+20]
 [-1.98546565e+20]
 [-1.36981296e+21]
 [-2.24355785e+20]
 [ 1.32779725e+21]]


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:22: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: RuntimeWarning: overflow encountered in exp
  
